# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [15]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

In [16]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [17]:
import os
print(os.getcwd())
role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))

/root/ImageClassificationAWSSageMakerPytorch
RoleArn: arn:aws:iam::546248296374:role/service-role/AmazonSageMaker-ExecutionRole-20240419T174807


In [32]:
#TODO: Fetch and upload the data to AWS S3

# !aws s3 cp /root/ImageClassificationAWSSageMakerPytorch/food_10_classes_10_percentimage.zip s3://fooddataset10class
# !aws s3 sync /root/ImageClassificationAWSSageMakerPytorch/food_10_classes_10_percent/train/ s3://fooddataset10class/train
# !aws s3 sync /root/ImageClassificationAWSSageMakerPytorch/food_10_classes_10_percent/test/ s3://fooddataset10class/test
# Command to download and unzip data
# !wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
# !aws s3 cp s3://fooddataset10class/food_10_classes_10_percentimage.zip .
# !unzip food_10_classes_10_percentimage.zip

In [18]:
BUCKET = 'fooddataset10class'

os.environ['SM_CHANNEL_TRAINING']=f's3://{BUCKET}/'
os.environ['SM_MODEL_DIR']=f's3://{BUCKET}/model/'
os.environ['SM_OUTPUT_DATA_DIR']=f's3://{BUCKET}/output/'

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [27]:
import sagemaker
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

In [28]:
#TODO: Declare your HP ranges, metrics etc.

hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "batch-size": CategoricalParameter([32, 64, 128]),
    "epochs":CategoricalParameter([10, 15, 20])
}

In [29]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="hpo.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.m5.large"
)

In [30]:
objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type=objective_type,
)

In [31]:
# TODO: Fit your HP Tuner
tuner.fit({"training":f"s3://{BUCKET}/"})

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................!


In [32]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()


2024-04-21 01:19:24 Starting - Found matching resource for reuse
2024-04-21 01:19:24 Downloading - Downloading the training image
2024-04-21 01:19:24 Training - Training image download completed. Training in progress.
2024-04-21 01:19:24 Uploading - Uploading generated training model
2024-04-21 01:19:24 Completed - Resource released due to keep alive period expiry


{'_tuning_objective_metric': '"average test loss"',
 'batch-size': '"128"',
 'epochs': '"15"',
 'lr': '0.00274723454500732',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"pytorch-training-2024-04-21-00-40-34-983"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-546248296374/pytorch-training-2024-04-21-00-40-34-983/source/sourcedir.tar.gz"'}

In [33]:
hyperparameters = {"batch-size": int(best_estimator.hyperparameters()['batch-size'].replace('"', '')), \
                   "lr": best_estimator.hyperparameters()['lr'],
                   "epochs": int(best_estimator.hyperparameters()['epochs'].replace('"', ''))
                  }
hyperparameters

{'batch-size': 128, 'lr': '0.00274723454500732', 'epochs': 15}

In [37]:
job_name = best_estimator.latest_training_job.name
client = estimator.sagemaker_session.sagemaker_client
description = client.describe_training_job(TrainingJobName=best_estimator.latest_training_job.name)
description

{'TrainingJobName': 'pytorch-training-240421-0040-004-10f49e5e',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:546248296374:training-job/pytorch-training-240421-0040-004-10f49e5e',
 'TuningJobArn': 'arn:aws:sagemaker:us-east-1:546248296374:hyper-parameter-tuning-job/pytorch-training-240421-0040',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-546248296374/pytorch-training-240421-0040-004-10f49e5e/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'_tuning_objective_metric': 'average test loss',
  'batch-size': '"128"',
  'epochs': '"15"',
  'lr': '0.00274723454500732',
  'sagemaker_container_log_level': '20',
  'sagemaker_estimator_class_name': '"PyTorch"',
  'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
  'sagemaker_job_name': '"pytorch-training-2024-04-21-00-40-34-983"',
  'sagemaker_program': '"hpo.py"',
  'sagemaker_region': '"us-east-1"',
  'sagemaker_submit_directory': '"s3://sagema

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

estimator = # TODO: Your estimator here

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()